In [3]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from io import StringIO

C:\Users\Chris\AppData\Local\Temp\ipykernel_2972\2900524488.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
requests.get?

Signature: requests.get(url, params=None, **kwargs)
Docstring:
Sends a GET request.

:param url: URL for the new :class:`Request` object.
:param params: (optional) Dictionary, list of tuples or bytes to send
    in the query string for the :class:`Request`.
:param \*\*kwargs: Optional arguments that ``request`` takes.
:return: :class:`Response <Response>` object
:rtype: requests.Response
File:      c:\users\chris\onedrive\documents\school\umaine fall 2022\sie 555\final project\baseball_canada_stats\env\lib\site-packages\requests\api.py
Type:      function

In [2]:
def scrape_batting_tables(url, html_text):
    # Concatenate the base URL with the print URL
    print_url = 'https://baseball.pointstreak.com/' + url
    print(f"Accessing print page: {print_url}")
    
    # Get the HTML content of the print page
    print_response = requests.get(print_url)
    # Use the pandas read_html function to read the tables from the HTML content
    dfs = pd.read_html(StringIO(print_response.text), attrs={'border': '1'})
    if dfs:
        # Get the first row of the dataframe and set it as the column names
        df = dfs[0]
        df.columns = df.iloc[0]
        # Remove the first row from the dataframe
        df = df[1:]
        # Remove any leading or trailing whitespaces from the column names and convert them to lowercase
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        # Filter out the 'Totals' row from the dataframe
        df = df[df.player != 'Totals']
        # Create soup object from the HTML content
        soup = BeautifulSoup(html_text, 'html.parser')
        # Find the div tag with the style attribute set to 'width:900px'
        div = soup.find('div', style='width:900px')
        # Find the h2 tag inside the div tag
        h2 = div.find('h2') if div else None
        # Get the title of the page from the h2 tag
        title = h2.string if h2 else 'unknown'
        # Replace any non-alphanumeric characters in the title with an underscore
        df_name = title.lower().replace(' ', '_') + '_df'
        # Save the dataframes as global variables
        globals()[df_name] = df
        print(f"Data stored in DataFrame: {df_name}")

def access_team_stats_pages(url):
    # Get the response from the URL
    response = requests.get(url)
    # Create a BeautifulSoup object from the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    # Find the ul tag with the class 'ps_teamlist list-unstyled'
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})
    # for each li tag in the ul tag team_list
    for li in team_list.find_all('li'):
        # find the a tag with the href attribute
        link = li.find('a', href=True)
        if link is not None:
            # Concatenate the base URL with the team URL
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing team page: {team_url}")
            # Get the response from the team URL
            team_response = requests.get(team_url)
            # Create a BeautifulSoup object from the HTML content
            team_soup = BeautifulSoup(team_response.text, 'html.parser')
            # Find the the vertical navigation bar on the team page
            nav = team_soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})
            if nav is not None:
                stats_link = None
                # for each li tag in the vertical navigation bar
                for item in nav.find_all('li', {'class': 'nova-list-item'}):
                    # Get the a tag with the href attribute
                    link = item.find('a', href=True)
                    # If the text of the a tag contains 'Stats' set the stats_link to the a tag
                    if link is not None and 'Stats' in link.text:
                        stats_link = link
                        break
                if stats_link is not None:
                    # Concatenate the base URL with the stats URL
                    stats_url = 'https://baseball.pointstreak.com/' + stats_link['href']
                    print(f"Accessing stats page: {stats_url}")
                    # Get the response from the stats URL
                    stats_response = requests.get(stats_url)
                    # Create a BeautifulSoup object from the HTML content
                    stats_soup = BeautifulSoup(stats_response.text, 'html.parser')
                    # Find the div tag with the class 'pull-left' (contains the print stats link)
                    print_div = stats_soup.find('div', {'class': 'pull-left'})
                    # Find the printable stats page
                    print_link = print_div.find('a', href=True, string='Print')
                    if print_link is not None:
                        # Concatenate the base URL with the print URL and get the response
                        print_response = requests.get('https://baseball.pointstreak.com/' + print_link['href'])
                        # Call the scrape_batting_tables function to scrape the batting tables
                        scrape_batting_tables(print_link['href'], print_response.text)

# Call the access_team_stats_pages function with the URL of the team stats page
access_team_stats_pages('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142674&seasonid=33653&view=batting
Data stored in DataFrame: edmonton_collegiate_batting_stats_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142669&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142669&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142669&seasonid=33653&view=batting
Data stored in DataFrame: okanagan_college_batting_stats_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142670&seasonid=33653
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142670&seasonid=33653
Accessing print page: https://baseball.pointstreak.com/pr

In [4]:
victoria_collegiate_batting_stats_df.head()

,player,p,avg,g,ab,r,h,2b,3b,hr,...,ops,slg,obp,pa,ab/hr,bb/pa,bb/k,seca,isop,rc
1,"Burton, T",OF,.297,31,101,27,30,4,2,1,...,.806,.406,.400,123,101,.106,.813,.366,.109,15.465
2,"Deagle, R",IF,.291,30,110,20,32,8,0,3,...,.868,.445,.423,137,36.667,.153,.84,.345,.155,19.824
3,"Biello, D",OF,.290,26,100,26,29,5,1,1,...,.808,.390,.418,122,100,.148,1.5,.3,.1,15.534
4,"Green, B",IF,.283,29,113,23,32,2,1,0,...,.718,.319,.399,138,0,.138,1.462,.23,.035,13.909
5,"Harris, P",C,.263,23,76,18,20,1,1,2,...,.799,.382,.417,96,38,.167,1.067,.355,.118,11.348


# With Pitching

In [29]:
def scrape_pitching_tables(stat_type, url, html_text):
    print_url = 'https://baseball.pointstreak.com/printstats.html' + url
    print(f"Accessing {stat_type} page: {print_url}")
    print_response = requests.get(print_url)
    dfs = pd.read_html(StringIO(print_response.text), attrs={'border': '1'})
    if dfs:
        df = dfs[0]
        df.columns = df.iloc[0]
        df = df[1:]
        df.columns = df.columns.str.lower().str.replace(' ', '_')
        df = df[df.player != 'Totals']
        # Create soup object from the HTML content
        soup = BeautifulSoup(print_response.text, 'html5lib')
        # Find the div tag with the style attribute set to 'width:900px'
        div = soup.find('div', style='width:900px')
        # Find the h2 tag inside the div tag
        h2 = div.find_all('h2')[0] if div else None
        title = h2.string if h2 else 'unknown'
        df_name = title.lower().replace(' ', '_') + '_df'
        globals()[df_name] = df
        print(f"Data stored in DataFrame: {df_name}")

def access_team_stats_pages(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    team_list = soup.find('ul', {'class': 'ps_teamlist list-unstyled'})

    for li in team_list.find_all('li'):
        link = li.find('a', href=True)
        if link is not None:
            team_url = 'https://baseball.pointstreak.com/' + link['href']
            print(f"Accessing team page: {team_url}")
            team_response = requests.get(team_url)
            print("Title at team page:", BeautifulSoup(team_response.text, 'html.parser').title.string)
            team_soup = BeautifulSoup(team_response.text, 'html.parser')
            nav = team_soup.find('ul', {'class': 'list-unstyled nav navbar-nav'})
            if nav is not None:
                stats_link = None
                for item in nav.find_all('li', {'class': 'nova-list-item'}):
                    link = item.find('a', href=True)
                    if link is not None and 'Stats' in link.text:
                        stats_link = link
                        break
                if stats_link is not None:
                    stats_url = 'https://baseball.pointstreak.com/' + stats_link['href']
                    print(f"Accessing stats page: {stats_url}")
                    stats_response = requests.get(stats_url)
                    print("Title at stats page:", BeautifulSoup(stats_response.text, 'html.parser').title.string)
                    stats_soup = BeautifulSoup(stats_response.text, 'html.parser')
                    div = stats_soup.find('div', style='margin-bottom:15px;')
                    if div is not None:
                        pitching_link = div.find('a', href=True, string='Pitching')
                        if pitching_link is not None:
                            pitching_url = 'https://baseball.pointstreak.com/team_stats.html' + pitching_link['href']
                            print(f"Accessing pitching page: {pitching_url}")
                            pitching_response = requests.get(pitching_url)
                            # TODO You did the print page for batting, but not for pitching (you did in the other function, but not here)
                            print("Title at pitching page:", BeautifulSoup(pitching_response.text, 'html.parser').title.string)
                            scrape_pitching_tables('print', pitching_link['href'], pitching_response.text)

access_team_stats_pages('https://baseball.pointstreak.com/teamlist.html?leagueid=160&seasonid=33653')

Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142674&seasonid=33653
Title at team page: Edmonton Collegiate - Canadian College Baseball Conference (CCBC) - team home | Pointstreak Sports Technologies
Accessing stats page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653
Title at stats page: Edmonton Collegiate - Canadian College Baseball Conference (CCBC) - team stats | Pointstreak Sports Technologies
Accessing pitching page: https://baseball.pointstreak.com/team_stats.html?teamid=142674&seasonid=33653&view=pitching
Title at pitching page: Edmonton Collegiate - Canadian College Baseball Conference (CCBC) - team stats | Pointstreak Sports Technologies
Accessing print page: https://baseball.pointstreak.com/printstats.html?teamid=142674&seasonid=33653&view=pitching
Data stored in DataFrame: edmonton_collegiate_pitching_stats_df
Accessing team page: https://baseball.pointstreak.com/team_home.html?teamid=142669&seasonid=33653
Title 

In [30]:
victoria_collegiate_pitching_stats_df.head()

,player,g,gs,cg,ip,h,r,er,bb,so,...,wp,hbp,whip,strike_%,pfr,bipa,k/9,bb/9,erc,fps%
1,"Merk, E",9,8,1,51.0,45,26,17,20,61,...,4,7,1.270,1.672,1.588,.268,10.765,3.529,3.1,0.493
2,"Puri, J",2,0,0,2.2,2,1,1,3,1,...,0,1,1.870,1,1.5,.167,3.375,10.125,8.591,0.500
3,"Seitz, T",9,0,0,23.1,16,15,9,14,24,...,5,4,1.290,1.387,1.629,.203,9.257,5.400,3.147,0.340
4,"Bolger, T",9,0,0,14.1,16,9,8,5,7,...,7,4,1.470,1.33,.837,.258,4.395,3.140,5.409,0.574
5,"Morgan, K",7,6,0,22.1,25,27,13,23,19,...,6,11,2.150,1.14,1.881,.229,7.657,9.269,9.247,0.376
